In [1]:
library(tidyverse)
library(caret)
library(glmnet)
library(dplyr)
#library(psych) 
library(lme4)
library(e1071)
library(GGally)
options(warn=-1)
options(scipen=999)
#library(crosstable)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

Warning message:
“package ‘tibble’ was built under R version 4.2.3”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘caret’ was built under R version 4.2.3”
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loaded glmnet 4.1-4

Warning message:
“package ‘lme4’ was built under R version 4.2.3”
Warning message:
“package ‘e1071’ was built under R version 4.2.3”
Warning message:
“package ‘GGally’ was built un

In [2]:
seed_list<-c(1001,1111,1221,1331,1441,1551,1661,1771,1881,1991)
cut_off<-c(0)
time_off<-c(10)
#time_off<-c(10)

lambda <- 10^seq(-3, 3, length = 100)
alpha<-seq(0,1,length=10)

In [3]:
library(doParallel)
cl <- makePSOCKcluster(20)
registerDoParallel(cl)

Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: iterators

Loading required package: parallel



In [4]:
m<-1
v<-1
#list_of_vi <- vector(mode="list", length=5)
#names(list_of_vi) <- c('10000', '10', '20', '50', '100')

list_of_vi_10000<-list()
list_of_vi_10<-list()
list_of_vi_20<-list()
list_of_vi_50<-list()
list_of_vi_100<-list()
list_of_rsqr=list()

corpus_list<-c("google")

ratings_list<-c("reddy","cordeiro90","cordeiro100")#,"reddy_true","cordeiro90_true","cordeiro100_true")
tagged_list<-c("Tagged")

ppmi_setting_list<-c("RAW")
comp_setting_list<-c("Aware")
impute_list<-c("med")
to_predict_list<-c("compound","modifier","head")

feature_setting_list<-c("all")#,"cordeiro","cosine_sim","with_setting","info_theory","freq","prod")
caret_spearman <- function(data, lev = NULL, model = NULL) {
  spearman_val <- cor(x = data$pred, y = data$obs, method = "spearman")
  c(Spearman = spearman_val)
}

needed_cols<-c('modifier','head','avgModifier','stdevModifier','avgHead','stdevHead','compositionality','stdevHeadModifier','is_adj','compound','source')

cordeiro_cols<-c('arith_mean_sim.','beta.','geom_mean_sim.','sim_cpf_0.','sim_cpf_100.','sim_cpf_25.','sim_cpf_50.','sim_cpf_75.','sim_cpf_beta.')
cosine_sim_cols<-c("sim_bw_constituents.",'sim_with_head.','sim_with_modifier.')

with_setting_cols<-c("sim_bw_settings_comp.","sim_bw_settings_head.","sim_bw_settings_modifier.")
info_theory_cols<-c("local_mi.","log_ratio.","ppmi.")
freq_cols<-c("comp_freq.","comp_tf.","head_freq.","head_tf.","log_comp_freq.","log_head_freq.","log_head_freq_new.","log_mod_freq.","log_mod_freq_new.","mod_freq.","mod_tf.")
prod_cols<-c("head_family_size.","head_family_size_new.","head_prod.","mod_family_size.","mod_family_size_new.","mod_prod.")


In [8]:
for (c in corpus_list){
  for (t in tagged_list) {
    
    for (p in ppmi_setting_list){
      for (a in comp_setting_list){
        for (i in time_off) {
          
          for (j in cut_off) {
            
            for (im in impute_list) {
              
              print(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_withSetting_",p,"_",t,"_",i,"_",j,"_",im,".csv")) 
              input_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_withSetting_",p,"_",t,"_",i,"_",j,"_",im,".csv"),sep = '\t')
              
              input_df <- input_df %>% distinct()
              for (r in ratings_list){
                
                print(dim( input_df))               
                
                
                reduced_df<-input_df %>% filter(source==r)
                reduced_df <- reduced_df %>% filter(!compound %in% c('call_centre','benign_tumour', 'computer_programme', 'grey_matter', 'labour_union','baby_blues', 'contact_lenses'))
                
                print(dim( reduced_df))               
                
                
                if (t=="Tagged") {
                  compounds_list<-c("an","nn","all")
                }
                else {
                  compounds_list<-c("all")
                  
                }
                for (k in compounds_list) {
                  df_features<-reduced_df
                  print(k)
                  if (k=="an") { 
                    group1_df<-df_features %>% filter(is_adj=="True" & grepl("_ADJ",modifier) & grepl("_NOUN",head))
                    group2_df<-df_features %>% filter(is_adj=="False" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    df_features<-rbind(group1_df,group2_df)                   
                  }
                  else if (k=="nn") {
                    group1_df<-df_features %>% filter(is_adj=="True" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    group2_df<-df_features %>% filter(is_adj=="False" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    df_features<-rbind(group1_df,group2_df)                    
                  }
                  
                  else if (k=="ap") {
                    group1_df<-df_features %>% filter(is_adj=="True" & grepl("_ADJ",modifier) & grepl("_PROPN",head))
                    group2_df<-df_features %>% filter(is_adj=="False" & grepl("_PROPN",modifier) & grepl("_PROPN",head))
                    df_features<-rbind(group1_df,group2_df)                    
                  }
                  
                  else if (k=="pp") {
                    group1_df<-df_features %>% filter(is_adj=="True" & grepl("_PROPN",modifier) & grepl("_PROPN",head))
                    group2_df<-df_features %>% filter(is_adj=="False" & grepl("_PROPN",modifier) & grepl("_PROPN",head))
                    df_features<-rbind(group1_df,group2_df)                    
                  }
                  print(dim( df_features))               
                  df_features_X<-df_features %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
                  print(dim( df_features_X)) 
                  df_features_X<- Filter(function(x) sd(x) != 0,df_features_X)
                  print(dim( df_features_X)) 
                  
                  for (f in feature_setting_list) {
                    
                    if(f=="cordeiro"){
                      trainX<-df_features_X %>% select(starts_with(cordeiro_cols))
                    }
                    else if(f=="cosine_sim"){
                      trainX<-df_features_X %>% select(starts_with(cosine_sim_cols))
                      
                    }
                    
                    else if(f=="with_setting"){
                      trainX<-df_features_X %>% select(starts_with(with_setting_cols))
                    }
                    
                    else if(f=="info_theory"){
                      trainX<-df_features_X %>% select(starts_with(info_theory_cols))
                    }
                    
                    else if(f=="freq"){
                      trainX<-df_features_X %>% select(starts_with(freq_cols))
                    }
                    else if(f=="prod"){
                      trainX<-df_features_X %>% select(starts_with(prod_cols))
                    }
                    else {
                      trainX<-df_features_X
                    }
                    
                    if (dim(trainX)[1]<10) {
                      break
                    }
                    for (pr in to_predict_list){
                      
                      if (pr=="compound") {
                        trainY<-df_features %>% select(compositionality)
                        trainY<-trainY$compositionality      
                      }
                      
                      else if (pr=="modifier") {
                        trainY<-df_features %>% select(avgModifier)
                        trainY<-trainY$avgModifier      
                      }
                      
                      else if (pr=="head") {
                        trainY<-df_features %>% select(avgHead)
                        trainY<-trainY$avgHead      
                      }                               
                      
                      print(paste0(c," ",t," ",p," ",a," ",i," ",j," ",im," ",r," ",k," ",f," ",pr))
                      
                      for (s in seed_list)  {
                        set.seed(s)
                        seeds <- vector(mode = "list", length = 14)
                        for(z in 1:13) seeds[[z]] <- sample.int(n=1000, 10)
                        #for the last model
                        seeds[[14]]<-sample.int(1000, 1)
                        
                        if (im=="na") {
                          preprocess_list<-c("nzv","medianImpute", "center", "scale")
                        }
                        else {
                          preprocess_list<-c("nzv", "center", "scale")
                        }
                        
                        
                        elastic_model <- train(trainX,trainY,method = "glmnet",metric = "Rsquared",
                                               trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                               preProcess = preprocess_list)
                        
                        elastic_spearman_model <- train(trainX,trainY,method = "glmnet",metric = "Spearman",
                                                        trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                                        preProcess = preprocess_list)
                        
                        
                        perf_elastic<-data.frame(corpus=c,tag=t,ppmi=p,setting=a,timespan=i,cutoff=j,impute=im,dataset=r,pattern=k,features=f,y=pr,n=nrow(trainX),seed=s,ml_algo="elastic",method=getTrainPerf(elastic_model)[,"method"],TrainRsquared=getTrainPerf(elastic_model)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_spearman_model)[,"TrainSpearman"])
                        
                        varimp_elastic<-data.frame(corpus=c,tag=t,ppmi=p,setting=a,timespan=i,cutoff=j,impute=im,dataset=r,pattern=k,features=f,y=pr,n=nrow(trainX),seed=s,ml_algo="elastic",t(varImp(elastic_model)$importance))
                        
                        
                        
                      }
                    }
                  }
                }
              }
            }
          }
        }
      }
    }
  }
}

[1] "/data/dharp/compounds/datasets/google/features_CompoundAware_withSetting_RAW_Tagged_10_0_med.csv"
[1] 1284  731
[1] 368 731
[1] "an"
[1]  90 731
[1]  90 720
[1]  90 700
[1] "google Tagged RAW Aware 10 0 med reddy an all compound"
[1] "google Tagged RAW Aware 10 0 med reddy an all modifier"


In [7]:
google_df <-read.csv(paste0("/data/dharp/compounds/datasets/features/features_CompoundAware_withSetting_PPMI_UnTagged_100_0_med.csv"),sep = '\t')

#google_df$compound_label<-cut_number(google_df$compositionality,n=3,labels=c("low","med","high"))
#google_df$corpus<-"google"

#combined_df<-rbind(coha_df,google_df)

#combined_df %>% filter(source=="reddy") %>%ggplot(aes(fill=compound_label,x=comp_tf.0)) + facet_grid(~corpus)+geom_density(alpha = 0.3) + theme_bw()

reddy_df<-google_df %>% filter(source=="reddy")

#cordeiro90_df<-google_df %>% filter(source=="cordeiro90")
#cordeiro100_df<-google_df %>% filter(source=="cordeiro100")

trainX<-reddy_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
trainY<-reddy_df %>% select(compositionality)
trainY<-trainY$compositionality


for (s in seed_list)  {
  set.seed(s)
  seeds <- vector(mode = "list", length = 14)
  for(z in 1:13) seeds[[z]] <- sample.int(n=1000, 10)
  #for the last model
  seeds[[14]]<-sample.int(1000, 1)

  ridge_nona_coha <- train(trainX,trainY,method = "glmnet",metric = "Rsquared",
                           trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                           preProc = c("nzv","center", "scale")
  )
  print(getTrainPerf(ridge_nona_coha)[,"TrainRsquared"])

}

[1] 0.5186674
[1] 0.5082243
[1] 0.4810678
[1] 0.4695517
[1] 0.5157097
[1] 0.5181739
[1] 0.4963162
[1] 0.53048
[1] 0.5125063
[1] 0.4958075
